# Using VeRA for sequence classification

In this example, we fine-tune Roberta on a sequence classification task using VeRA.

In [1]:
!git clone https://github.com/MQN-80/mindnlp
!cd mindnlp && git checkout jinyi
#!pip install -r requirements.txt
!pip uninstall transformers && pip install tokenizers==0.19.1
#!pip uninstall peft -y
!cd mindnlp && bash scripts/build_and_reinstall.sh

流式输出内容被截断，只能显示最后 5000 行内容。
        - https://setuptools.pypa.io/en/latest/userguide/package_discovery.html

        If you don't want 'mindnlp.quant.mindbnb.csrc' to be distributed and are
        already explicitly excluding 'mindnlp.quant.mindbnb.csrc' via
        `find_namespace_packages(...)/find_namespace` or `find_packages(...)/find`,
        you can try to use `exclude_package_data`, or `include-package-data=False` in
        combination with a more fine grained `package-data` configuration.

        You can read more about "package data files" on setuptools documentation page:

        - https://setuptools.pypa.io/en/latest/userguide/datafiles.html


        [^1]: For Python, any directory (with suitable naming) can be imported,
              even if it does not contain any `.py` files.
              On the other hand, currently there is no concept of package data
              directory, all directories are treated like packages.
        ***************************************

## Imports

In [8]:
import mindspore
from mindnlp.core.optim import AdamW
#from torch.utils.data import DataLoader
from mindnlp.peft import LoraConfig, TaskType, get_peft_model
from mindnlp.peft import VeraConfig,LoKrConfig

import evaluate
from mindnlp.dataset import load_dataset
#from datasets import load_dataset
from mindnlp.transformers import AutoModelForSequenceClassification, AutoConfig,AutoTokenizer
#from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed, AutoConfig
from tqdm import tqdm

## Parameters

In [3]:
batch_size = 128
model_name_or_path = "roberta-base"
task = "mrpc"
#peft_type = PeftType.VERA
#device = "cuda"
num_epochs = 5  # for best results, increase this number
rank = 8        # for best results, increase this number
max_length = 128
#torch.manual_seed(0)

In [4]:
# peft_config = VeraConfig(
#     task_type="SEQ_CLS",
#     r=rank,
#     d_initial=0.1,
#     target_modules=["query", "value", "intermediate.dense"],
#     save_projection=True,
# )
peft_config = VeraConfig(
    task_type="SEQ_CLS",
    r=2,
    d_initial=0.1,
    target_modules=["query", "value", "intermediate.dense"],
    save_projection=True,
)
head_lr = 1e-2
vera_lr = 2e-2

## Loading data

In [7]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

  0%|          | 0.00/25.0 [00:00<?, ?B/s]

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [21]:
datasets = load_dataset("glue", task)
ds = load_dataset('rotten_tomatoes', split='train')
metric = evaluate.load("glue", task)

{'train': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset object at 0x793540aacf90>, 'validation': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset object at 0x79354095e9d0>, 'test': <mindspore.dataset.engine.datasets_user_defined.GeneratorDataset object at 0x793548d0fa90>}


TypeError: missing a required argument: 'operations'

In [31]:
def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    print(examples)
    outputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, max_length=max_length)
    return outputs
tokenized_datasets = datasets['train'].map(
    operations=tokenize_function,
    #remove_columns=["idx", "sentence1", "sentence2"],
)
# We also rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
tokenized_datasets = tokenized_datasets.rename("label", "labels")

In [ ]:
def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

## Preparing the VeRA model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True, max_length=None)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

  0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 647,498 || all params: 125,294,668 || trainable%: 0.5167801713637168


In [ ]:
optimizer = AdamW(
    [
        {"params": [p for n, p in model.named_parameters() if "vera_lambda_" in n], "lr": vera_lr},
        {"params": [p for n, p in model.named_parameters() if "classifier" in n], "lr": head_lr},
    ]
)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

## Training

In [ ]:
model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|██████████| 4/4 [00:01<00:00,  3.52it/s]


epoch 0: {'accuracy': 0.7475490196078431, 'f1': 0.8367670364500792}


100%|██████████| 4/4 [00:01<00:00,  3.37it/s]


epoch 1: {'accuracy': 0.7671568627450981, 'f1': 0.8536209553158706}


100%|██████████| 4/4 [00:01<00:00,  3.33it/s]


epoch 2: {'accuracy': 0.8553921568627451, 'f1': 0.8959435626102292}


100%|██████████| 4/4 [00:01<00:00,  3.35it/s]


epoch 3: {'accuracy': 0.8823529411764706, 'f1': 0.9133574007220215}


100%|██████████| 4/4 [00:01<00:00,  3.17it/s]

epoch 4: {'accuracy': 0.8897058823529411, 'f1': 0.9183303085299456}


## Share adapters on the 🤗 Hub

In [ ]:
account_id = ...  # your Hugging Face Hub account ID

In [ ]:
model.push_to_hub(f"{account_id}/roberta-large-peft-vera")

CommitInfo(commit_url='https://huggingface.co/BenjaminB/roberta-large-peft-vera/commit/d720cdc67b97df9cd1453b14d3e0fb17efc8779b', commit_message='Upload model', commit_description='', oid='d720cdc67b97df9cd1453b14d3e0fb17efc8779b', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer

In [ ]:
peft_model_id = f"{account_id}/roberta-large-peft-vera"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load the Vera model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id)

In [ ]:
inference_model.to(device)
inference_model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = inference_model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

eval_metric = metric.compute()
print(eval_metric)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.14it/s]

{'accuracy': 0.8480392156862745, 'f1': 0.8938356164383561}
